In [4]:
import math
import random
from scipy.optimize import brentq

def ellipse_perimeter_ramanujan(a, b):
    """
    Returns Ramanujan's approximate perimeter of an ellipse
    with semi-axes a and b.
    """
    # If a == b (circle), perimeter = 2 * pi * a
    if abs(a - b) < 1e-15:
        return 2.0 * math.pi * a
    
    h = ((a - b)**2) / ((a + b)**2)
    perimeter = math.pi * (a + b) * (
        1.0 + (3.0 * h) / (10.0 + math.sqrt(4.0 - 3.0 * h))
    )
    return perimeter

def solve_for_a_scipy(b, C, tol=1e-7):
    def g(a):
        return ellipse_perimeter_ramanujan(a, b) - C
    
    a_low = max(b, 1e-12)  # to avoid zero if b=0
    g_low = g(a_low)
    
    # (A) If we are already at the solution or extremely close:
    if abs(g_low) < tol:
        return a_low  # a = b basically solves it.
    
    # (B) If perimeter at a=b is larger than target, no solution for a>=b
    if g_low > 0:
        # Because perimeter grows with a, there's no a >= b that reduces perimeter
        raise ValueError("No solution with a >= b: perimeter at b is already above target.")
    
    # We proceed only if g_low < 0 (meaning perimeter(b) < C).
    # Next, bracket upward:
    a_high = a_low
    while True:
        a_high *= 2.0
        g_high = g(a_high)
        if g_high > 0:
            # sign change => bracket found
            break
        if a_high > 1e15:
            raise ValueError("Could not bracket the solution up to a=1e15. Possibly no valid solution.")
    
    # Now call brentq
    a_solution = brentq(g, a_low, a_high, xtol=tol, rtol=1e-12)
    return a_solution


def run_tests():
    """
    Runs a series of test cases to verify correctness and robustness of
    solve_for_a(b, C). Prints out pass/fail messages and details.
    """
    
    # Helper to check difference
    def check_close(val, ref, eps=1e-6):
        return abs(val - ref) < eps
    
    # ------------------------------------------------------------
    # 1. Test exact circle: a = b
    #    If we set a=b, the perimeter is 2 * pi * b.
    #    Then we pass that perimeter to solve_for_a.
    # ------------------------------------------------------------
    print("TEST 1: Circle tests (a=b)")
    for b_test in [1.0, 5.0, 10.0, 123.456]:
        C_circle = 2.0 * math.pi * b_test
        a_solution = solve_for_a_scipy(b_test, C_circle)
        # We expect a_solution to be extremely close to b_test
        diff = abs(a_solution - b_test)
        perimeter_recalc = ellipse_perimeter_ramanujan(a_solution, b_test)
        print(f"  b={b_test:8.3f}, target C={C_circle:10.5f}  ->  solved a={a_solution:10.5f}, diff(a-b)={diff:.3g}")
        print(f"     Recalc perimeter={perimeter_recalc:.6f}, error={abs(perimeter_recalc - C_circle):.3g}")
        print("     PASS" if (diff < 1e-5) else "     FAIL")
    print()
    
    # ------------------------------------------------------------
    # 2. Test moderate ellipse where a>b, but not huge difference
    #    We'll pick a few pairs (a, b), compute C, and see if we get back ~a.
    # ------------------------------------------------------------
    print("TEST 2: Moderate ellipses (a ~ 1-3x b)")
    test_pairs = [(6, 4), (8, 5), (10, 5), (3.5, 3), (12, 10)]
    for (a_true, b_test) in test_pairs:
        C_ellipse = ellipse_perimeter_ramanujan(a_true, b_test)
        a_solution = solve_for_a_scipy(b_test, C_ellipse)
        diff = abs(a_solution - a_true)
        perimeter_recalc = ellipse_perimeter_ramanujan(a_solution, b_test)
        err_perim = abs(perimeter_recalc - C_ellipse)
        print(f"  True (a,b)=({a_true},{b_test}), perimeter={C_ellipse:.5f}")
        print(f"    -> Solved a={a_solution:.5f}, error in a={diff:.3g}, perimeter check error={err_perim:.3g}")
        print("    PASS" if (diff < 1e-4 and err_perim < 1e-4) else "    FAIL")
    print()
    
    # ------------------------------------------------------------
    # 3. Test very elongated ellipses (a >> b).
    #    If a is very large vs b, perimeter ~ 4a (but not exactly).
    # ------------------------------------------------------------
    print("TEST 3: Very elongated ellipses (a >> b)")
    test_pairs_elon = [
        (100, 1),
        (1000, 10),
        (1e4, 100),
    ]
    for (a_true, b_test) in test_pairs_elon:
        C_ellipse = ellipse_perimeter_ramanujan(a_true, b_test)
        a_solution = solve_for_a_scipy(b_test, C_ellipse)
        diff = abs(a_solution - a_true)
        perimeter_recalc = ellipse_perimeter_ramanujan(a_solution, b_test)
        err_perim = abs(perimeter_recalc - C_ellipse)
        print(f"  True (a,b)=({a_true},{b_test}), perimeter={C_ellipse:.5f}")
        print(f"    -> Solved a={a_solution:.5f}, error in a={diff:.3g}, perimeter check error={err_perim:.3g}")
        print("    PASS" if (diff < 1e-3 and err_perim < 1e-3) else "    FAIL")
    print()
    
    # ------------------------------------------------------------
    # 4. Test random ellipses within a certain range
    # ------------------------------------------------------------
    print("TEST 4: Random ellipses")
    random.seed(1234)  # for reproducibility
    for _ in range(5):
        b_rand = random.uniform(1.0, 50.0)
        a_rand = random.uniform(b_rand, b_rand*5)  # ensure a >= b but up to 5x
        C_rand = ellipse_perimeter_ramanujan(a_rand, b_rand)
        a_solution = solve_for_a_scipy(b_rand, C_rand)
        diff = abs(a_solution - a_rand)
        recalc = ellipse_perimeter_ramanujan(a_solution, b_rand)
        err_perim = abs(recalc - C_rand)
        print(f"  Random ellipse (a_true={a_rand:.3f}, b={b_rand:.3f}) => perimeter={C_rand:.3f}")
        print(f"    -> a_solved={a_solution:.3f}, diff in a={diff:.3g}, perimeter err={err_perim:.3g}")
        print("    PASS" if (diff < 1e-4 and err_perim < 1e-4) else "    FAIL")
    print()
    
    # ------------------------------------------------------------
    # 5. Test extremely close to a circle but not exactly
    # ------------------------------------------------------------
    print("TEST 5: Ellipse extremely close to circle (a ~ b)")
    b_val = 10.0
    # Let a be just slightly bigger, e.g. 10.0001
    a_close = 10.0001
    C_close = ellipse_perimeter_ramanujan(a_close, b_val)
    a_sol = solve_for_a_scipy(b_val, C_close)
    diff_close = abs(a_sol - a_close)
    perimeter_close = ellipse_perimeter_ramanujan(a_sol, b_val)
    perimeter_err = abs(perimeter_close - C_close)
    print(f"  a={a_close}, b={b_val}, perimeter={C_close:.6f}")
    print(f"    -> solved a={a_sol}, diff in a={diff_close:.2e}")
    print(f"       perimeter recalc={perimeter_close:.6f}, diff={perimeter_err:.2e}")
    print("    PASS" if (diff_close < 1e-5) else "    FAIL")

# -------------
# MAIN EXECUTION
# -------------
if __name__ == "__main__":
    run_tests()


TEST 1: Circle tests (a=b)
  b=   1.000, target C=   6.28319  ->  solved a=   1.00000, diff(a-b)=0
     Recalc perimeter=6.283185, error=0
     PASS
  b=   5.000, target C=  31.41593  ->  solved a=   5.00000, diff(a-b)=0
     Recalc perimeter=31.415927, error=0
     PASS
  b=  10.000, target C=  62.83185  ->  solved a=  10.00000, diff(a-b)=0
     Recalc perimeter=62.831853, error=0
     PASS
  b= 123.456, target C= 775.69693  ->  solved a= 123.45600, diff(a-b)=0
     Recalc perimeter=775.696925, error=0
     PASS

TEST 2: Moderate ellipses (a ~ 1-3x b)
  True (a,b)=(6,4), perimeter=31.73088
    -> Solved a=6.00000, error in a=3.49e-12, perimeter check error=1.2e-11
    PASS
  True (a,b)=(8,5), perimeter=41.38628
    -> Solved a=8.00000, error in a=2.18e-12, perimeter check error=7.57e-12
    PASS
  True (a,b)=(10,5), perimeter=48.44224
    -> Solved a=10.00000, error in a=7.44e-10, perimeter check error=2.66e-09
    PASS
  True (a,b)=(3.5,3), perimeter=20.45057
    -> Solved a=3.50000,